In [92]:
import numpy as np

class Dual:
    def __init__(self, real, dual={}):
        self.__real = real
        self.__dual = dual


    def __add__(self, other):
        res = Dual(0)
        res.__dual = self.__dual
        if isinstance(other, Dual):
            res.__real = self.__real + other.__real
            for k, v in other.__dual.items():
                if k in res.__dual:
                    res.__dual[k] += v
                else:
                    res.__dual[k] = v
        else:
            res.__real = self.__real + other
        return res

    def __radd__(self, other):
        res = Dual(0)
        res.__dual = self.__dual
        if isinstance(other, Dual):
            res.__real = self.__real + other.__real
            for k, v in other.__dual.items():
                if k in res.__dual:
                    res.__dual[k] += v
                else:
                    res.__dual[k] = v
        else:
            res.__real = self.__real + other
        return res

    def __sub__(self, other):
        res = Dual(0)
        res.__dual = self.__dual
        if isinstance(other, Dual):
            res.__real = self.__real - other.__real
            for k, v in other.__dual.items():
                if k in res.__dual:
                    res.__dual[k] -= v
                else:
                    res.__dual[k] = -v
        else:
            res.__real = self.__real - other
        return res

    def __rsub__(self, other):
        res = Dual(0)
        res.__dual = other.__dual
        if isinstance(other, Dual):
            res.__real = -self.__real + other.__real
            for k, v in self.__dual.items():
                if k in res.__dual:
                    res.__dual[k] -= v
                else:
                    res.__dual[k] = -v
        else:
            res.__real = -self.__real + other
        return res

    def __mul__(self, other):
        if isinstance(other, Dual):
            real = self.__real * other.__real
            dual = {k: v * other.__real for k, v in self.__dual.items()}
            for key in other.__dual:
                if key in dual:
                    dual[key] += other.__dual[key] * self.__real
                else:
                    dual[key] = other.__dual[key] * self.__real
            return Dual(real, dual)
        else:
            dual = {}
            for key in self.__dual:
                dual[key] = self.__dual[key] * other
            return Dual(self.__real * other, dual)

    def __rmul__(self, other):
        if isinstance(other, Dual):
            real = self.__real * other.__real
            dual = {k: v * other.__real for k, v in self.__dual.items()}
            for key in other.__dual:
                if key in dual:
                    dual[key] += other.__dual[key] * self.__real
                else:
                    dual[key] = other.__dual[key] * self.__real
            return Dual(real, dual)
        else:
            dual = {}
            for key in self.__dual:
                dual[key] = self.__dual[key] * other
            return Dual(self.__real * other, dual)

    def __truediv__(self, other):
        denominator = pow(other.__real, 2)
        res = Dual(None)
        if isinstance(other, Dual):
            res.__real = self.__real / other.__real
            new_other = other.__neg_dual()
            tmp = self * new_other
            for k, v in tmp.__dual.items():
                res.__dual[k] = v / denominator
        else:
            res.__real = self.__real / other
            for k, v in self.__dual.items():
                res.__dual[k] = v / other
        return res

    def __rtruediv__(self, other):
        denominator = pow(other.__real, 2)
        tmp = other * self.__neg_dual()
        res = Dual(None)
        res.__real = tmp.__real / denominator
        for k, v in tmp.__dual.items():
            res.__dual[k] = v / denominator
        return res

    def __pow__(self, power):
        res = Dual(None)
        real = pow(self.__real, power)
        res.__real = real
        dual = {}
        for k, v in self.__dual.items():
            dual[k] = power * v * (pow(self.__real, power - 1))
        res.__dual = dual
        return res

    def __neg__(self):
        res = Dual(None)
        res.__real = -self.__real
        res.__dual = self.__dual
        for k, v, in self.__dual.items():
            res[k] = -v
        return res

    # Следует из разложения в ряд Тейлора
    def sin(self):
        dual = {}
        real = self.__real
        for k, v in self.__dual.items():
            dual[k] = v * np.cos(real)
        return Dual(np.sin(real), dual)

    def cos(self):
        dual = {}
        real = self.__real
        for k, v in self.__dual.items():
            dual[k] = v * -np.sin(real)
        return Dual(np.cos(real), dual)

    def exp(self):
        dual = {}
        real = self.__real
        for k, v in self.__dual.items():
            dual[k] = v * np.exp(real)
        return Dual(np.exp(real), dual)

    def log(self):
        dual = {}
        real = self.__real
        for k, v in self.__dual.items():
            dual[k] = v / real
        return Dual(np.log(real), dual)


    def derivative(self, diff_var_name):
        return self.__dual.get(diff_var_name)

    def __neg_dual(self):
        dual = {}
        for k, v in self.__dual.items():
            dual[k] = -v
        return Dual(self.__real, dual)

    def __call__(self, *args, **kwargs):
        return self.__real

    @property
    def dual(self):
        return self.__dual

x = Dual(np.pi, {'x': 1})
y = Dual(np.pi, {'y': 1})

f = x.sin() / (y.cos() + x ** 2)
print(f.derivative('y'))

1.9063963744630746e-34
